In [58]:
import pandas as pd
import numpy as np

# import stats functions from scipy
from scipy import stats

# imports for better control of output and plots
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

# show plots in the notebook
%matplotlib inline

DATA_PATH = '../data/KaggleV2-May-2016.csv'
df = pd.read_csv(DATA_PATH)
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


zmiana wartości na categorical + usunięcie apointment ID bo jest unikatowe dla kazdej wizyty i nie niesie żadnej informacji
Zamiana na datatime object

In [59]:
df.drop('AppointmentID', axis=1,inplace = True)
df['PatientId'] = df['PatientId'].apply(int).apply(str)
df['No-show'] = df['No-show'].map({'No':0, 'Yes':1})
df['Gender'] = df['Gender'].map({'F':0, 'M':1})
df['Handcap'] = pd.Categorical(df['Handcap'])
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay']).dt.strftime('%Y-%m-%d')
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay'])
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay']).dt.strftime('%Y-%m-%d')
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'])
df.head()

,PatientId,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,29872499824296,0,2016-04-29,2016-04-29,62,JARDIM DA PENHA,0,1,0,0,0,0,0
1,558997776694438,1,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,0,0,0,0,0,0
2,4262962299951,0,2016-04-29,2016-04-29,62,MATA DA PRAIA,0,0,0,0,0,0,0
3,867951213174,0,2016-04-29,2016-04-29,8,PONTAL DE CAMBURI,0,0,0,0,0,0,0
4,8841186448183,0,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,1,1,0,0,0,0


Nowa wartość -> Sprawdzenie ile razy pacjent ominął juz wizytę

In [60]:
df['Num_App_Missed'] = df.groupby('PatientId')['No-show'].apply(lambda x: x.cumsum())
df.head()

,PatientId,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,Num_App_Missed
0,29872499824296,0,2016-04-29,2016-04-29,62,JARDIM DA PENHA,0,1,0,0,0,0,0,0
1,558997776694438,1,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,0,0,0,0,0,0,0
2,4262962299951,0,2016-04-29,2016-04-29,62,MATA DA PRAIA,0,0,0,0,0,0,0,0
3,867951213174,0,2016-04-29,2016-04-29,8,PONTAL DE CAMBURI,0,0,0,0,0,0,0,0
4,8841186448183,0,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,1,1,0,0,0,0,0


Nowa wartość -> różnica w dniach miedzy odbytą wizytą a zaplanowaną wizytą

In [61]:
df['Day_diff'] = (df['AppointmentDay'] - df['ScheduledDay']).dt.days
df['Day_diff'].unique()


array([  0,   2,   3,   1,   4,   9,  29,  10,  23,  11,  18,  17,  14,
        28,  24,  21,  15,  16,  22,  43,  30,  31,  42,  32,  56,  45,
        46,  39,  37,  38,  44,  50,  60,  52,  53,  65,  67,  91,  66,
        84,  78,  87, 115, 109,  63,  70,  72,  57,  58,  51,  59,  41,
        49,  73,  64,  20,  33,  34,   6,  35,  36,  12,  13,  40,  47,
         8,   5,   7,  25,  26,  48,  27,  19,  61,  55,  62, 176,  54,
        77,  69,  83,  76,  89,  81, 103,  79,  68,  75,  85, 112,  -1,
        80,  86,  98,  94, 142, 155, 162, 169, 104, 133, 125,  96,  88,
        90, 151, 126, 127, 111, 119,  74,  71,  82, 108, 110, 102, 122,
       101, 105,  92,  97,  93, 107,  95,  -6, 139, 132, 179, 117, 146,
       123])

In [62]:
df[df.Age < 0] 
df[df.Day_diff < 0]

,PatientId,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,Num_App_Missed,Day_diff
27033,7839272661752,1,2016-05-10,2016-05-09,38,RESISTÊNCIA,0,0,0,0,1,0,1,1,-1
55226,7896293967868,0,2016-05-18,2016-05-17,19,SANTO ANTÔNIO,0,0,0,0,1,0,1,2,-1
64175,24252258389979,0,2016-05-05,2016-05-04,22,CONSOLAÇÃO,0,0,0,0,0,0,1,2,-1
71533,998231581612122,0,2016-05-11,2016-05-05,81,SANTO ANTÔNIO,0,0,0,0,0,0,1,2,-6
72362,3787481966821,1,2016-05-04,2016-05-03,7,TABUAZEIRO,0,0,0,0,0,0,1,1,-1


Usunięcie tych dziwnych danych, gdzie wiek mniejszy od zera i wizyty które odbywały się szybciej niż zaplanowano (?)

In [63]:
df = df[(df.Age > 0) & (df.Age < 105)] 
df = df[df.Day_diff >= 0]
df.head()

,PatientId,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,Num_App_Missed,Day_diff
0,29872499824296,0,2016-04-29,2016-04-29,62,JARDIM DA PENHA,0,1,0,0,0,0,0,0,0
1,558997776694438,1,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,0,0,0,0,0,0,0,0
2,4262962299951,0,2016-04-29,2016-04-29,62,MATA DA PRAIA,0,0,0,0,0,0,0,0,0
3,867951213174,0,2016-04-29,2016-04-29,8,PONTAL DE CAMBURI,0,0,0,0,0,0,0,0,0
4,8841186448183,0,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,1,1,0,0,0,0,0,0


In [64]:
df.Handcap.unique()

[0, 1, 2, 3, 4]
Categories (5, int64): [0, 1, 2, 3, 4]

Zamieńmy Handcamp na kategorie binarne

In [65]:
Handicap = pd.get_dummies(df['Handcap'], prefix = 'Handicap')
df = pd.concat([df, Handicap], axis=1)
df.drop(['Handcap'], axis=1, inplace = True)
df.head()

,PatientId,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,SMS_received,No-show,Num_App_Missed,Day_diff,Handicap_0,Handicap_1,Handicap_2,Handicap_3,Handicap_4
0,29872499824296,0,2016-04-29,2016-04-29,62,JARDIM DA PENHA,0,1,0,0,0,0,0,0,1,0,0,0,0
1,558997776694438,1,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,0,0,0,0,0,0,0,1,0,0,0,0
2,4262962299951,0,2016-04-29,2016-04-29,62,MATA DA PRAIA,0,0,0,0,0,0,0,0,1,0,0,0,0
3,867951213174,0,2016-04-29,2016-04-29,8,PONTAL DE CAMBURI,0,0,0,0,0,0,0,0,1,0,0,0,0
4,8841186448183,0,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,1,1,0,0,0,0,0,1,0,0,0,0


Usuńmy zbędne kolumny

In [66]:
df.drop(['ScheduledDay'], axis=1, inplace=True)
df.drop(['AppointmentDay'], axis=1, inplace=True)
df.drop('PatientId', axis=1,inplace = True)
df.drop('Neighbourhood', axis=1,inplace = True)

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106977 entries, 0 to 110526
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype
---  ------          --------------   -----
 0   Gender          106977 non-null  int64
 1   Age             106977 non-null  int64
 2   Scholarship     106977 non-null  int64
 3   Hipertension    106977 non-null  int64
 4   Diabetes        106977 non-null  int64
 5   Alcoholism      106977 non-null  int64
 6   SMS_received    106977 non-null  int64
 7   No-show         106977 non-null  int64
 8   Num_App_Missed  106977 non-null  int64
 9   Day_diff        106977 non-null  int64
 10  Handicap_0      106977 non-null  uint8
 11  Handicap_1      106977 non-null  uint8
 12  Handicap_2      106977 non-null  uint8
 13  Handicap_3      106977 non-null  uint8
 14  Handicap_4      106977 non-null  uint8
dtypes: int64(10), uint8(5)
memory usage: 9.5 MB


In [68]:
df.head()

,Gender,Age,Scholarship,Hipertension,Diabetes,Alcoholism,SMS_received,No-show,Num_App_Missed,Day_diff,Handicap_0,Handicap_1,Handicap_2,Handicap_3,Handicap_4
0,0,62,0,1,0,0,0,0,0,0,1,0,0,0,0
1,1,56,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,62,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,8,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,56,0,1,1,0,0,0,0,0,1,0,0,0,0


In [69]:
# Zmienna niezależna na koniec
noshow = df['No-show']
df.drop('No-show', axis=1,inplace = True)
df = pd.concat([df, noshow], axis=1)
df.head()

,Gender,Age,Scholarship,Hipertension,Diabetes,Alcoholism,SMS_received,Num_App_Missed,Day_diff,Handicap_0,Handicap_1,Handicap_2,Handicap_3,Handicap_4,No-show
0,0,62,0,1,0,0,0,0,0,1,0,0,0,0,0
1,1,56,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,62,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,8,0,0,0,0,0,0,0,1,0,0,0,0,0
4,0,56,0,1,1,0,0,0,0,1,0,0,0,0,0


In [73]:
# Min Max scaler dla zmiennej Age i Num_app_Missed and Day_diff
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[['Age', 'Num_App_Missed', 'Day_diff']] = scaler.fit_transform(df[['Age', 'Num_App_Missed', 'Day_diff']])
df.head()


,Gender,Age,Scholarship,Hipertension,Diabetes,Alcoholism,SMS_received,Num_App_Missed,Day_diff,Handicap_0,Handicap_1,Handicap_2,Handicap_3,Handicap_4,No-show
0,0,0.603960,0,1,0,0,0,0.0,0.0,1,0,0,0,0,0
1,1,0.544554,0,0,0,0,0,0.0,0.0,1,0,0,0,0,0
2,0,0.603960,0,0,0,0,0,0.0,0.0,1,0,0,0,0,0
3,0,0.069307,0,0,0,0,0,0.0,0.0,1,0,0,0,0,0
4,0,0.544554,0,1,1,0,0,0.0,0.0,1,0,0,0,0,0


In [77]:
df['Num_App_Missed'].unique()

array([0.        , 0.05555556, 0.11111111, 0.16666667, 0.22222222,
       0.27777778, 0.33333333, 0.38888889, 0.44444444, 0.5       ,
       0.55555556, 0.61111111, 0.66666667, 0.72222222, 0.77777778,
       0.83333333, 0.88888889, 0.94444444, 1.        ])

In [74]:
X = np.array(df.iloc[:, :-1].values)
y = np.array(df.iloc[:,-1].values)
check_y = list(y)
print(f'Procent wizyt odwołanych: {round(check_y.count(1) / len(check_y),3)} %, procent wizyt nie odwołanych {round(check_y.count(0) / len(check_y),3)} %')
print(f'Liczba wizyt odwołanych {check_y.count(1)}')
not_cancelled = np.where(y == 0)
not_cancelled = not_cancelled[0]

import random
not_cancelled = np.array(random.sample(list(not_cancelled), check_y.count(1)))
print(f'Wizyty nie odwołane liczba: {len(not_cancelled)}')

# get indexes for canceled visits
cancelled = np.where(y == 1)[0]
print(f'Wizyty odwołane liczba: {len(cancelled)}')

indexes = np.concatenate((not_cancelled, cancelled))
# Balance dataset
X = X[indexes]
y = y[indexes]

print('Po przetworzeniu')
check_y = list(y)
print(f'Procent wizyt odwołanych: {round(check_y.count(1) / len(check_y),3)} %, procent wizyt nie odwołanych {round(check_y.count(0) / len(check_y),3)} %')
print(f'Liczba wizyt odwołanych {check_y.count(1)}')

Procent wizyt odwołanych: 0.203 %, procent wizyt nie odwołanych 0.797 %
Liczba wizyt odwołanych 21672
Wizyty nie odwołane liczba: 21672
Wizyty odwołane liczba: 21672
Po przetworzeniu
Procent wizyt odwołanych: 0.5 %, procent wizyt nie odwołanych 0.5 %
Liczba wizyt odwołanych 21672


## Testowanie modeli

In [78]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

def test_model(X, y, typ):
    
    # Create training and testing samples
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    # Fit the model
    model = typ
    clf = model.fit(X_train, y_train)

    # Predict class labels on a test data
    pred_labels = model.predict(X_test)

    # Use score method to get accuracy of the model
    print('--------------------------------------------------------')
    score = model.score(X_test, y_test)
    print('Accuracy Score: ', score)
    print('--------------------------------------------------------')
    
    # Look at classification report to evaluate the model
    print(classification_report(y_test, pred_labels))
    
    # Return relevant data for chart plotting
    return X_train, X_test, y_train, y_test, clf, pred_labels

### Naive Bayes

In [79]:
from sklearn.naive_bayes import GaussianNB
X_train, X_test, y_train, y_test, clf, y_pred, = test_model(X, y, GaussianNB())

--------------------------------------------------------
Accuracy Score:  0.8510785557734456
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.88      0.82      0.85      4395
           1       0.82      0.89      0.85      4274

    accuracy                           0.85      8669
   macro avg       0.85      0.85      0.85      8669
weighted avg       0.85      0.85      0.85      8669



In [81]:
from sklearn.model_selection import cross_val_score

accuracy = cross_val_score(estimator = GaussianNB(), X = X, y =y, cv = 8)
print("avg acc: ",np.mean(accuracy))
print("avg std: ",np.std(accuracy))

avg acc:  0.767326504245109
avg std:  0.13835398374290467


## KNN

In [82]:
from sklearn.neighbors import KNeighborsClassifier
X_train, X_test, y_train, y_test, clf, y_pred, = test_model(X, y, KNeighborsClassifier())

--------------------------------------------------------
Accuracy Score:  0.9160226092974968
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.95      0.88      0.91      4395
           1       0.88      0.95      0.92      4274

    accuracy                           0.92      8669
   macro avg       0.92      0.92      0.92      8669
weighted avg       0.92      0.92      0.92      8669



In [83]:
accuracy = cross_val_score(estimator = KNeighborsClassifier(), X = X, y =y, cv = 8)
print("avg acc: ",np.mean(accuracy))
print("avg std: ",np.std(accuracy))

avg acc:  0.9066998892580289
avg std:  0.008031974522669445


## RFC

In [84]:
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test, clf, y_pred, = test_model(X, y, RandomForestClassifier())

--------------------------------------------------------
Accuracy Score:  0.9245587726381359
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.94      0.91      0.92      4395
           1       0.91      0.94      0.92      4274

    accuracy                           0.92      8669
   macro avg       0.92      0.92      0.92      8669
weighted avg       0.93      0.92      0.92      8669



In [85]:
accuracy = cross_val_score(estimator = RandomForestClassifier(), X = X, y =y, cv = 8)
print("avg acc: ",np.mean(accuracy))
print("avg std: ",np.std(accuracy))

avg acc:  0.9216269841269842
avg std:  0.00632063667755573
